In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

In [ ]:
df = pd.read_excel("data/MONEYDEM-1.xls")
df['year'] = df['DATE'].astype(int)
df['quarter'] = ((df['DATE'] - df['year']) * 10).round().astype(int)
df['date'] = df['year'].astype(str) + 'Q' + df['quarter'].astype(str)
df['date'] = pd.PeriodIndex(df['date'], freq='Q')
df.set_index('date', inplace=True)
df

In [ ]:
#1a 
plt.figure(figsize=(8, 5))
plt.plot(df.index.to_timestamp(), df['TB3mo'], marker='o')
plt.title('Quarterly Data')
plt.xlabel('Date')
plt.ylabel('TB3mo')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(df.index.to_timestamp(), df['TB1yr'], marker='o')
plt.title('Quarterly Data')
plt.xlabel('Date')
plt.ylabel('TB1yr')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# No, they appear to have changing mean and variance given that in 1980 had a vilont spike 

In [ ]:
#1b
plt.figure(figsize=(8, 5))
plt.plot(df.index.to_timestamp(), df['TB1yr'], marker='o')
plt.plot(df.index.to_timestamp(), df['TB3mo'], marker='o')
plt.title('Quarterly Data')
plt.xlabel('Date')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# They apear to be practicaly identical the seam to follow the same trend

In [ ]:
#1c
model = smf.ols("TB1yr ~ TB3mo", data=df).fit()
print(model.summary())

In [ ]:
#1d
# A 1 percentage point increase in the 3-month Treasury rate (short-term) is associated with a 0.9167 percentage point increase in the 1-year Treasury rate (long-term), on average, holding other factors constant.

In [ ]:
#e
t_test_result = model.t_test('TB3mo = 1')
print(t_test_result)

In [ ]:
# since p-value is 0.000 there is strong but less-than-perfect pass-through from short- to long-term rates.


In [ ]:
#f
residuals = model.resid

# Plot residuals vs TB3mo
plt.figure(figsize=(8, 5))
plt.scatter(df['TB3mo'], residuals, alpha=0.7)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('TB3mo (3-month Treasury rate)')
plt.ylabel('Residuals')
plt.title('Residuals vs TB3mo')
plt.grid(True)
plt.show()


In [ ]:
# No there does not seam to be a pattern 

In [ ]:
#g
residuals = model.resid
exog = model.model.exog 

white_test = sms.het_white(residuals, exog)

lm_stat, lm_pvalue, f_stat, f_pvalue = white_test

print(f"White test LM statistic: {lm_stat:.4f}")
print(f"White test LM p-value: {lm_pvalue:.4f}")
print(f"White test F statistic: {f_stat:.4f}")
print(f"White test F p-value: {f_pvalue:.4f}")


In [ ]:
# given that the p-value is 0.0000 there is evidence of heteroskedasticity

In [ ]:
#1h
model_robust = smf.ols("TB1yr ~ TB3mo", data=df).fit(cov_type='HC0')
print(model_robust.summary())

In [ ]:
#h
# the coefecients did not change, however the std err did increase for both the intercept 
# and the coeficient 
# When heteroskedasticity is present, regular SEs underestimate the true variability of the coefficients, so the robust SEs tend to be larger and more reliable.

In [ ]:
#j
df['D'] = (df['TB3mo'] > 10.00).astype(int)
model_with_dummy = smf.ols("TB1yr ~ TB3mo + D", data=df).fit()

print(model_with_dummy.summary())

In [ ]:
# 1k
# given that the p-value of delta is 0.004 the dummy is revelat. 

In [ ]:
# 1l
# Test the null that the dummy variable is relevant in part (j).